In [42]:
from transformers import AutoImageProcessor, TimesformerForVideoClassification
import numpy as np
import torch

video = np.random.rand(4,8, 3, 224, 224)

processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k600")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k600")

inputs = processor(images=video, return_tensors="pt")

with torch.no_grad():
  outputs = model.timesformer(**inputs).last_hidden_state
  feature = outputs[:,1:].reshape(-1, 8, 14, 14, 768)
  

print(feature.shape)

ValueError: Unsupported number of image dimensions: 5

In [ ]:
from transformers import BeitForSemanticSegmentation, BeitConfig
config = BeitConfig.from_pretrained("microsoft/beit-base-finetuned-ade-640-640")
config.num_channels = 9
config.image_size = 640
backbone = BeitForSemanticSegmentation.from_pretrained("microsoft/beit-base-finetuned-ade-640-640", config=config, ignore_mismatched_sizes=True)
backbone.decode_head.classifier = torch.nn.Identity()


/home/wg25r/.local/lib/python3.10/site-packages/transformers/models/beit/configuration_beit.py:197: FutureWarning: The `segmentation_indices` argument is deprecated and will be removed in a future version, use `out_indices` instead.
  warnings.warn(
Some weights of BeitForSemanticSegmentation were not initialized from the model checkpoint at microsoft/beit-base-finetuned-ade-640-640 and are newly initialized because the shapes did not match:
- beit.embeddings.patch_embeddings.projection.weight: found shape torch.Size([768, 3, 16, 16]) in the checkpoint and torch.Size([768, 9, 16, 16]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
import torch 
backbone(torch.randn(1, 9, 640, 640)).logits.shape

torch.Size([1, 768, 160, 160])

In [28]:
from transformers import SegformerForSemanticSegmentation
backbone = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b5-finetuned-cityscapes-1024-1024")
backbone

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b5-finetuned-cityscapes-1024-1024 and are newly initialized: ['segformer.encoder.test.bias', 'segformer.encoder.test.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [2]:
import torchvision
swin_t = torchvision.models.swin_t()

In [6]:
import torch
swin_t.features[1][0](torch.randn(1, 224, 224, 96)).shape

torch.Size([1, 224, 224, 96])